In [ ]:
# If training in collab
! pip install cloud-tpu-client https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.13-cp38-cp38m-linux_x86_64.whl

In [ ]:
! pip install lightning tensorboard

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import lightning as L
import os
from matplotlib import pyplot as plt
import torch.utils.data as data

In [2]:
tfs = transforms.Compose([
    transforms.ColorJitter(brightness=0.05, contrast=0.05, saturation=0.05, hue=0.05),
    transforms.ToTensor
])


teacher_train_set = MNIST(os.getcwd(), download=True, train=True, transform=tfs)
student_train_set = MNIST(os.getcwd(), download=True, train=True, transform=transforms.ToTensor())
test_set = MNIST(os.getcwd(), download=True, train=False, transform=transforms.ToTensor())

train_set_size = int(len(teacher_train_set) * 0.8)
valid_set_size = len(teacher_train_set) - train_set_size

# split the train set into two
seed = torch.Generator().manual_seed(42)
train_set, valid_set = data.random_split(teacher_train_set, [train_set_size, valid_set_size], generator=seed)
train_loader = DataLoader(train_set)
validation_loader = DataLoader(valid_set)

In [3]:
from teacher import MnistTeacher as Teacher
from student import MnistStudent as Student
from distillation import Distillation as Distiller

In [4]:
from lightning.pytorch.loggers import TensorBoardLogger

# model
teacher = Teacher()

# train model
trainer = L.Trainer(
            logger=TensorBoardLogger("tb_logs", name="teacher"),
            max_epochs=10
        )
trainer.fit(teacher, train_loader, validation_loader)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: tb_logs/teacher

  | Name | Type       | Params
------------------------------------
0 | net  | Sequential | 2.4 M 
------------------------------------
2.4 M     Trainable params
0         Non-trainable params
2.4 M     Total params
9.581     Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/anaconda3/envs/pytorch-lightning/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


/usr/local/anaconda3/envs/pytorch-lightning/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Epoch 1:  86%|████████▌ | 41352/48000 [33:30<05:23, 20.56it/s, v_num=0]

In [ ]:
distiller = Distiller()

# train model
trainer = L.Trainer(
            logger=TensorBoardLogger("tb_logs", name="teacher"),
            max_epochs=10
        )
trainer.fit(model=distiller, train_dataloaders=train_loader)